In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys

Append the `grt_reader` repo so we can read modules from sibling directories.


In [3]:
sys.path.append(os.path.abspath(os.path.join("..")))

In [4]:
import numpy as np
import pandas as pd
from utils.constants import question_ids_map
from utils.questions import (
    construct_single_question_for_ai,
)

from notebooks.utils.process_lease import process_lease

/Users/david/grt/grt_reader/.venv/lib/python3.11/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [5]:
bucket_name = os.getenv("BUCKET_NAME", "")
index_name = os.getenv("INDEX_NAME", "")

In [6]:
print(f"Bucket name: {bucket_name}")
print(f"Index name: {index_name}")

Bucket name: gaaprt-reader-dev
Index name: development-gaaprt-reader-openai


Retrieve questions


In [7]:
questions_df = pd.read_excel("../data/asc_842/grt_ai_reader_questions_v1.xlsx")
questions_df = questions_df.drop(["encoding"], axis=1)
questions_df = questions_df.replace({np.nan: None})

Retrieve answers file


In [8]:
answers_df = pd.read_excel("../data/asc_842/lease_agreements_info/answers.xlsx")

Get questions to send to the AI


In [9]:
question_id = question_ids_map["option_to_extend"]
question_df_filtered = questions_df[questions_df["id"] == question_id]
question_as_json = question_df_filtered.to_dict(orient="records")[0]
question_for_ai = construct_single_question_for_ai(question_as_json)

In [10]:
file_name = "lease001"
local_file = f"../data/asc_842/lease_agreements/{file_name}.pdf"
file_key = f"eafit/{file_name}.pdf"
namespace = f"eafit_{file_name}"

In [14]:
result = process_lease(
    local_file=local_file,
    file_name=file_name,
    file_key=file_key,
    namespace=namespace,
    question_for_ai=question_for_ai,
    question_id=question_id,
    answers_df=answers_df,
    model="gpt-4o-mini",
    embedding_model="text-embedding-3-small",
)

Upload Successful: eafit/lease001.pdf
Stored eafit/lease001.pdf in vector database with namespace eafit_lease001


In [15]:
answer = result["answer"]
real_answer_unprocessed = result["real_answer_unprocessed"]

In [16]:
answer

QuestionResponse(question_id='32', answer_string=None, answer_boolean='0', answer_date=None, meta='The lease agreement states that the term shall expire on the five (5) year anniversary of the Rent Commencement Date, indicating there is no option to extend the lease beyond this period.')

In [15]:
float(answer)

0.0

In [16]:
real_answer_unprocessed.iloc[0]

0.0

In [17]:
float(answer) == real_answer_unprocessed.iloc[0]

True